In [3]:
import re
import os, datetime
from glob import glob
import numpy as np
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract
from keras.models import Model, load_model
from keras.optimizers import Adam
import keras.backend as K
import cv2
from tensorflow import distributions as dist
import logging
import warnings
warnings.filterwarnings('ignore')
from numbers import Number
import tensorflow as tf

Using TensorFlow backend.


In [69]:
def normcdf(value, mu=0.0, stddev=1.0):
    sinv = (1.0 / stddev) if isinstance(stddev, Number) else stddev.reciprocal()
    return 0.5 * (1.0 + tf.math.erf((value - mu) * sinv / np.sqrt(2.0)))

def _normal_log_pdf(value, mu, stddev):
    var = (stddev ** 2)
    log_scale = np.log(stddev) if isinstance(stddev, Number) else torch.log(stddev)
    return -((value - mu) ** 2) / (2.0*var) - log_scale - np.log(np.sqrt(2.0*np.pi))


def normpdf(value, mu=0.0, stddev=1.0):
    return tf.math.exp(_normal_log_pdf(value, mu, stddev))


def _lpdn_relu_activation(m, v):
#     v = tf.maximum(v, 0.0001)
#     v = tf.math.exp(v)
#     s = v ** 0.5
#     u_out = u * normal.cdf(u / s) + s * normal.prob(u / s)
#     v_out = (u + v) * normal.cdf(u / s) + (u * s) * normal.prob(u / s) - u_out ** 2
#     v_out = tf.math.log(v_out)
    
    stddev = tf.math.sqrt(v)
    div = m/stddev
    pdf = normpdf(div)
    cdf = normcdf(div)
    out_m = m*cdf + stddev*pdf
    out_v = (m ** 2 + v)*cdf + m*stddev*pdf - out_m ** 2
    out_v = tf.math.log(out_v)
    return out_m, out_v


class LPActivation(Activation): ### forward
    def __init__(self, activation, **kwargs):
        if isinstance(activation, str):
            self.activation_name = activation
        else:
            self.activation_name = activation.__name__
        super(LPActivation, self).__init__(activation, **kwargs)

    def call(self, inputs):
        u = inputs[...,0]
        v = inputs[...,1] ## log
        u, v = _lpdn_relu_activation(u, v)
        v = tf.math.log(v)
        return tf.stack([u,v],-1)

In [ ]:
class LPConv2D(Conv2D):
    """
    Propagate distributions over a probabilistic Conv2D layer
    """

    def __init__(self, filters, kernel_size, **kwargs):
        super(LPConv2D, self).__init__(filters, kernel_size, **kwargs)

    def build(self, input_shape):
        return super(LPConv2D, self).build(input_shape[:-1])

    def compute_output_shape(self, input_shape):
        original_output_shape = super(LPConv2D, self).compute_output_shape(input_shape[:-1])
        return original_output_shape + (2,)

    def assert_input_compatibility(self, inputs):
        return super(LPConv2D, self).assert_input_compatibility(inputs[..., 0])

    def _conv2d(self, input, kernel):
        return K.conv2d(input, kernel, self.strides, self.padding, self.data_format, self.dilation_rate)

    def call(self, inputs): ### forward
        u = inputs[...,0]
        v = inputs[...,1]
        
        v = tf.math.exp(v)
        
        u = self._conv2d(u, self.kernel)
        v = self._conv2d(v, self.kernel ** 2)
#         print(self.kernel)
        v = tf.math.log(v)
        if self.use_bias:
            u = K.bias_add(
                u,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            u, v = _lpdn_relu_activation(u, v)      
            v = tf.math.log(v)
            
        return tf.stack([u,v],-1)


In [ ]:
def data_aug(img, mode=0):

    if mode == 0:
        return img
    elif mode == 1:
        return np.flipud(img)
    elif mode == 2:
        return np.rot90(img)
    elif mode == 3:
        return np.flipud(np.rot90(img))
    elif mode == 4:
        return np.rot90(img, k=2)
    elif mode == 5:
        return np.flipud(np.rot90(img, k=2))
    elif mode == 6:
        return np.rot90(img, k=3)
    elif mode == 7:
        return np.flipud(np.rot90(img, k=3))


def datagenerator(data_dir = 'data_set/train_clean/*.png',
                  data_dir_noisy = 'data_set/train_noisy/*.png'
                  ,verbose=False):
    
    file_list = glob(data_dir)
    file_list2 = glob(data_dir_noisy)
    # initrialize
    data = []
    data2 = []
    # generate patches
    for i in range(len(file_list)):
        img = cv2.imread(file_list[i],0)
        img_noisy = cv2.imread(file_list2[i],0)
        img = np.array(img)
        img_noisy = np.array(img_noisy)
        img = np.reshape(img,(180,180,1))
        img_noisy = np.reshape(img_noisy,(180,180,1))
        final_clean = np.zeros((180,180,1,2))
        final_noisy = np.zeros((180,180,1,2))
        final_clean[...,0] = img
        final_clean[...,1] = img
        final_noisy[...,0] = img_noisy
        final_noisy[...,1] = img_noisy
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        data.append(final_clean)
        data2.append(final_noisy)
    data = np.array(data,dtype = 'float32')
    data_noisy = np.array(data2,dtype = 'float32')
    print('^_^-training data finished-^_^')
    return data,data_noisy

def train_datagen(epoch_iter=20,epoch_num=1,batch_size=10,data_dir='data_set/train_clean/*.png'):
    while(True):
        n_count = 0
        if n_count == 0:
            xs,xs2 = datagenerator(data_dir)
            xs = xs.astype('float32')/255.0
            xs2 = xs.astype('float32')/255.0
            indices = list(range(xs.shape[0]))
            n_count = 1
        for _ in range(epoch_num):
            np.random.shuffle(indices)    # shuffle
            for i in range(0, len(indices), batch_size):
                batch_x = xs[indices[i:i+batch_size]]
                batch_y = xs2[indices[i:i+batch_size]]
                yield batch_y, batch_x


In [ ]:
def DnCNN(depth, filters, image_channels):
    layer_count = 0
    inpt = Input(shape=(None, None, image_channels), name='input' + str(layer_count))
    # 1st layer, Conv+relu
    layer_count += 1
    x1 = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), kernel_initializer='Orthogonal', padding='same',
               name='conv' + str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu', name='relu' + str(layer_count))(x1)
    # depth-2 layers, Conv+relu
    for i in range(depth - 2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), kernel_initializer='Orthogonal', padding='same',
                   use_bias=False, name='conv' + str(layer_count))(x)
        layer_count += 1
        x = Activation('relu', name='relu' + str(layer_count))(x)
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3, 3), strides=(1, 1), kernel_initializer='Orthogonal',
               padding='same', use_bias=False, name='conv' + str(layer_count))(x)
    
    layer_count += 1
    model = Model(inputs=inpt, outputs= x)
    
    return model

In [ ]:
def convert(keras_model, input_shape=None):
    # Create an equivalent probabilistic model.
    if input_shape is None:
        input_shape = keras_model.layers[0].input_shape[1:] + (2,)
        logging.info("Inferred input shape: " + str(input_shape))

    lp_input = Input(shape=input_shape)
    y = lp_input
    for li, l in enumerate(keras_model.layers):
        if isinstance(l, Conv2D):
            y = LPConv2D(l.filters, l.kernel_size, padding=l.padding, activation=l.activation, name=l.name)(y)
        elif isinstance(l, Activation):
            y = LPActivation(l.activation, name=l.name)(y)
    model = Model(inputs=lp_input, outputs=y)
#     y[...,0] = model.get_layer('conv1').output[...,0] - model.get_layer('conv33').output[...,0]
#     model2  = Model(inputs = lp_input,outputs = y)
    
    return model

In [ ]:
def ADF_loss(y_true, y_pred):
    mean = y_pred[...,0]
    variance = tf.maximum(0.0001,tf.math.exp(y_pred[...,1]))
    #print(variance.shape)
    eps = 1e-5
    temp = 1/(variance + eps)
    output = K.sum(temp*(y_true[...,0] - mean)**2 + tf.math.log(variance + eps))
    return output

In [92]:
model = DnCNN(depth=17,filters=64,image_channels=1)
model = convert(model)
model.compile(optimizer=Adam(0.001), loss=ADF_loss)
history = model.fit_generator(train_datagen(batch_size=5),
                steps_per_epoch=20, epochs=5, verbose=1)

<tf.Variable 'conv1_26/kernel:0' shape=(3, 3, 1, 64) dtype=float32>
<tf.Variable 'conv3_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv5_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv7_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv9_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv11_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv13_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv15_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv17_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv19_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv21_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv23_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv25_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv27_23/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
<tf.Variable 'conv29_23/ke

KeyboardInterrupt: 

In [76]:
a = (3,3)